In [3]:
import pandas as pd
import unicodedata

In [4]:
url_componentes = 'componentes-graduacao.csv'
url_unidades = 'http://dados.ufrn.br/dataset/da6451a5-1a59-4630-bdc2-97f6be4a59c2/resource/3f2e4e32-ef1a-4396-8037-cbc22a89d97f/download/unidades.csv'
url_docentes = 'http://dados.ufrn.br/dataset/8bf1a468-48ff-4f4d-95ee-b17b7a3a5592/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070/download/docentes.csv'
url_avaliacoes = 'http://dados.ufrn.br/dataset/d5723d75-7e6e-4264-82aa-b96909b69f63/resource/7accd1d2-2793-460e-b98d-87a0679b9155/download/avaliacaodocencia.csv'

In [5]:
turma_periodo = ['http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e6e4144f-4042-4fdc-84e0-76e9ec27ae7c/download/turmas-2014.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/2c69547b-920f-4ec2-92c0-3fbc19512165/download/turmas-2014.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/0d96d930-8058-4def-9044-c3ae04c1f40c/download/turmas-2013.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e7e3cf12-a29b-491a-a895-021a43819197/download/turmas-2013.2.csv',]

In [6]:
frames = [pd.read_csv(periodo, sep=';', usecols=['id_turma', 'ano', 'nivel_ensino', 'periodo', 'id_componente_curricular']) for periodo in turma_periodo]

In [7]:
df_turmas = pd.concat(frames)

In [8]:
df_turmas.rename(columns={'nivel_ensino': 'nivel',}, inplace=True)
df_turmas = df_turmas[df_turmas['nivel'] == 'GRADUAÇÃO']
df_turmas['periodo'] = df_turmas['periodo'].astype(int) 
df_turmas.head()

,id_turma,id_componente_curricular,nivel,ano,periodo
158,57593679,24946.0,GRADUAÇÃO,2017,1
159,57593680,24945.0,GRADUAÇÃO,2017,1
160,57593681,2041855.0,GRADUAÇÃO,2017,1
161,57593682,26579.0,GRADUAÇÃO,2017,1
162,57593683,2041856.0,GRADUAÇÃO,2017,1


In [9]:
df_unidades = pd.read_csv(url_unidades, ';')
df_unidades = df_unidades[['id_unidade_responsavel', 'unidade_responsavel']]
df_unidades.rename(columns={'id_unidade_responsavel': 'id_unidade', 'unidade_responsavel': 'lotacao'}, inplace=True)
df_unidades = df_unidades[(df_unidades['lotacao'] == 'DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA') | (df_unidades['lotacao'] == 'INSTITUTO METROPOLE DIGITAL')].drop_duplicates()
df_unidades.reset_index(inplace=True, drop=True)
df_unidades.head()

,id_unidade,lotacao
0,125,DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA
1,6069,INSTITUTO METROPOLE DIGITAL


In [10]:
df_docentes = pd.read_csv(url_docentes, ';')
df_docentes.columns

Index([u'id_servidor', u'siape', u'nome', u'formacao',
       u'tipo_jornada_trabalho', u'vinculo', u'categoria', u'classe_funcional',
       u'id_unidade_lotacao', u'lotacao', u'admissao', u'Unnamed: 11'],
      dtype='object')

In [11]:
df_docentes = df_docentes[df_docentes['categoria'] == 'PROFESSOR DO MAGISTERIO SUPERIOR']

In [12]:
df_docentes = df_docentes[['id_servidor', 'nome', 'formacao', 'id_unidade_lotacao', 'admissao']]
df_docentes.rename(columns={'id_servidor': 'id_docente', 'admissao': 'dt_admissao', 'id_unidade_lotacao': 'id_unidade'}, inplace=True)
df_docentes.head()

,id_docente,nome,formacao,id_unidade,dt_admissao
205,5752188,ABDO FARRET NETO,DOUTORADO,427,2009-02-09 00:00:00
206,16930,ABMAEL BEZERRA DE OLIVEIRA,MESTRADO,112,1973-03-01 00:00:00
207,5758354,ABRAHAO SANDERSON NUNES FERNANDES DA SILVA,DOUTORADO,5486,2016-04-08 00:00:00
208,23906,ADA CRISTINA SCUDELARI,DOUTORADO,52,1997-04-22 00:00:00
209,5754609,ADAILDO GOMES D ASSUNCAO,DOUTORADO,5204,2011-12-01 00:00:00


In [14]:

df_docentes[df_docentes['id_unidade'] == 125]

,id_docente,nome,formacao,id_unidade,dt_admissao
384,24949,ANDRE MAURICIO CUNHA CAMPOS,DOUTORADO,125,2002-10-08 00:00:00
409,24853,ANNE MAGALY DE PAULA CANUTO,DOUTORADO,125,2002-04-25 00:00:00
419,5756592,ANTONIO CARLOS GAY THOME,DOUTORADO,125,2014-03-05 00:00:00
454,5755408,AUGUSTO JOSE VENANCIO NETO,DOUTORADO,125,2013-01-03 00:00:00
464,26044,BENJAMIN RENE CALLEJAS BEDREGAL,DOUTORADO,125,1999-02-26 00:00:00
485,25978,BRUNO MOTTA DE CARVALHO,DOUTORADO,125,2004-07-28 00:00:00
515,5755388,CARLOS AUGUSTO PROLO,DOUTORADO,125,2013-01-14 00:00:00
697,5752467,EDGARD DE FARIA CORREA,DOUTORADO,125,2009-03-27 00:00:00
724,5752241,EDUARDO HENRIQUE DA SILVA ARANHA,DOUTORADO,125,2009-01-27 00:00:00
759,23713,ELIZABETH FERREIRA GOUVEA GOLDBARG,DOUTORADO,125,1996-06-13 00:00:00


In [2]:
df_componentes = pd.read_csv(url_componentes, sep=',')
df_componentes = df_componentes[(df_componentes['unidade_responsavel'] == 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA') | (df_componentes['unidade_responsavel'] == 'INSTITUTO METROPOLE DIGITAL')]
df_componentes.head()

NameError: name 'pd' is not defined

In [12]:
unidades_normalizadas = [unicodedata.normalize('NFKD', unicode(nome, "utf-8")).encode('ASCII', 'ignore') for nome in df_componentes['unidade_responsavel']]
series = pd.Series(data=unidades_normalizadas, index=df_componentes.index)
df_componentes['unidade_responsavel'] = series

In [13]:
df_componentes = df_componentes.merge(df_unidades[['id_unidade', 'lotacao']], left_on='unidade_responsavel', right_on='lotacao', how='inner')
df_componentes[df_componentes['codigo'] == 'IMD0504']

,id_componente,codigo,nivel,nome,unidade_responsavel,id_unidade,lotacao
376,52692,IMD0504,G,"RELAÇÕES ENTRE COMUNICAÇÃO, MÍDIA E EDUCAÇÃO",INSTITUTO METROPOLE DIGITAL,6069,INSTITUTO METROPOLE DIGITAL


In [14]:
df_componente_curricular = df_componentes[['id_componente', 'codigo']]
df_componente_curricular.head()

,id_componente,codigo
0,20007,DIM0072
1,21073,DIM0056
2,21083,DIM0346
3,21667,DIM0046
4,21668,DIM0003


In [15]:
df_componente_lotacao = df_componentes[['codigo', 'id_unidade', 'nome']]
df_componente_lotacao.head()

,codigo,id_unidade,nome
0,DIM0072,125,TECNICAS DE ESPECIFICACAO FORMAL
1,DIM0056,125,SOFTWARE BASICO
2,DIM0346,125,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...
3,DIM0046,125,ALGORITMOS E TECNICAS DE PROGRAMACAO
4,DIM0003,125,ORGANIZACAO DE COMPUTADORES


In [16]:
df_avaliacoes = pd.read_csv(url_avaliacoes, sep=';', usecols=['id_docente', 'id_turma', 'qtd_discentes', 'postura_profissional_media', 'postura_profissional_DP', 'atuacao_profissional_media', 'atuacao_profissional_DP'])
df_avaliacoes.rename(columns={'postura_profissional_media': 'postura_profissional', 'atuacao_profissional_media': 'atuacao_profissional'}, inplace=True)
df_avaliacoes.head()

,id_docente,id_turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP
0,15370,1209923,14,9.17,1.34,8.13,2.70
1,15370,1222008,15,9.44,0.92,7.17,3.08
2,15370,1222218,38,9.19,1.67,6.77,3.29
3,15370,1229520,13,9.06,1.16,7.68,2.51
4,15370,1229545,24,9.27,1.28,8.45,1.72


In [17]:
df_avaliacoes.to_csv('avaliacao.csv', sep=';', index=False)
df_componente_lotacao.to_csv('componente_lotacao.csv', sep=';', index=False)
df_componente_curricular.to_csv('componente_curricular.csv', sep=';', index=False)
df_docentes.to_csv('docente.csv', sep=';', index=False)
df_unidades.to_csv('unidade.csv', sep=';', index=False)
df_turmas.to_csv('turma.csv', sep=';', index=False)